Overall Steps

In [ ]:
# Get hugging face dataset - Done
# Resize all images to 448 by 448 - Done
# For each image, split into grid of 16 x 16 pixels - sequences of each patch representing the image - Done
# Convert each patch to 1 dimensional vector - this will be the pre embedding of the patch - Done
# Pass pre embedding to embedding layer to get the embedding of the patch 
# This will input to the encoder 

# Tokenise each caption in the dataset - Done
# Create two copies of each tokenised caption, 
#  - one with start of sentence token at the start - input to decoder
#  - one with end of sentence token at the end - label for the loss function 

In [19]:
from PIL import Image
import numpy as np
import torch

from tqdm import tqdm
import pandas as pd
import sentencepiece as spm
import ast  # For converting string representation of list to list
import torch.nn as nn
import math



Split up image into grid of 16 x 16 and convert to list of 1D arrays

In [28]:

# Path to your image
image_path = 'flickr30k/flickr30k-images/testImage.jpg'


def convertImageTo1DPatchesVectorList(image_path):
    
    # Load the image
    image = Image.open(image_path)

    # Resize the image to 448 x 448 pixels
    resized_image = image.resize((448, 448))

    # Convert the image to a numpy array for easier manipulation
    image_np = np.array(resized_image)

    # Initialize a list to hold the 1D vectors for each 16x16 patch
    patch_vectors = []

    # Iterate over the image in 16x16 blocks
    for y in range(0, image_np.shape[0], 16):
        for x in range(0, image_np.shape[1], 16):
            # Extract the patch
            patch = image_np[y:y+16, x:x+16]

            # Flatten the patch to a 1D vector (16*16*number_of_channels)
            # This line assumes the image is in color (RGB), so the patch size becomes 16*16*3
            patch_vector = patch.flatten()
            
            # Add the patch vector to our list
            patch_vectors.append(patch_vector)

        # Convert the list of patch vectors to a single numpy array
    patch_vectors_np = np.array(patch_vectors)

     # Convert the numpy array of patch vectors to a PyTorch tensor
    patch_vectors_tensor = torch.tensor(patch_vectors_np, dtype=torch.float32)
    
    return patch_vectors_tensor

patch_vectors_tensor = convertImageTo1DPatchesVectorList(image_path)

print(f"Total patches extracted: {len(patch_vectors_tensor)}")
# Verify the shape
print("Shape of patch vectors tensor:", patch_vectors_tensor.shape)



Total patches extracted: 784
Shape of patch vectors tensor: torch.Size([784, 768])


Sentencepiece sub word encoding

In [7]:
# Tokenise the captions 
# Create the copies (one for the decoder, the second for the loss function label)


def encode_sentences(csv_path, sp_model_path):
    # Load the SentencePiece model
    sp = spm.SentencePieceProcessor(model_file=sp_model_path)
    
    # Read the CSV file
    df = pd.read_csv(csv_path)
    
    # Prepare a dictionary to hold encoded sentences
    encoded_sentences = {'img_id': [], 'encoded_sentences': []}
    
    for _, row in df.iterrows():
        # Convert string representation of list to actual list
        sentences = ast.literal_eval(row['raw'])
        
        # Encode each sentence in the list
        encoded = [sp.encode(sentence, out_type=int) for sentence in sentences]
        
        # Append results
        encoded_sentences['img_id'].append(row['img_id'])
        encoded_sentences['encoded_sentences'].append(encoded)
    
    # Convert dictionary to DataFrame for easy handling/viewing
    encoded_df = pd.DataFrame(encoded_sentences)
    return encoded_df

# Usage example
csv_path = 'flickr_annotations_30k.csv'
sp_model_path = 'spm.model'
encoded_captions = encode_sentences(csv_path, sp_model_path) #Dimensions - batch size x num_images x num_captions x max caption length (after padding)

31014


In [26]:
encoded_captions.encoded_sentences[0] #encoding for 5 captions 

[[19, 29, 373, 14, 2280, 124, 195, 20, 74, 177, 30, 360, 83, 7, 8, 538, 5],
 [19, 29, 15, 1113, 792, 17, 62, 95, 367, 1554, 5],
 [19, 40, 7, 55, 281, 17, 37, 7, 4, 538, 5],
 [6, 12, 7, 4, 32, 25, 37, 7, 4, 805, 5],
 [19, 555, 573, 653, 3793, 1927, 157, 5]]

Positional Encoding

In [20]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_length):
        super().__init__()
        self.pos_encoding = self.create_pos_encoding(d_model, max_length)

    def create_pos_encoding(self, d_model, max_length):
        position = torch.arange(max_length).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model))
        pe = torch.zeros(max_length, d_model)
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        return nn.Parameter(pe, requires_grad=False)
    
    def forward(self, embeddings):
        # Use embeddings to determine sequence length and to get device information
        seq_len = embeddings.size(1)
        pos_encoding = self.pos_encoding[:, :seq_len, :].to(embeddings.device)
        return embeddings + pos_encoding

Image Encoder


In [31]:
class ModifiedImageEncoder(nn.Module):
    def __init__(self, num_patches, embed_dim=768, num_layers=6, num_heads=12, d_ff=2048, dropout=0.1):
        super().__init__()
        self.num_patches = num_patches
        self.embed_dim = embed_dim
        self.positional_encoding = PositionalEncoding(embed_dim, num_patches)
        
        encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads, dim_feedforward=d_ff, dropout=dropout, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

    def forward(self, flat_patches):
        # Assume flat_patches is of shape [B, num_patches, embed_dim]
        # where each patch is already flattened into a vector of embed_dim
        x = flat_patches
        
        # You might need to adjust positional encodings if the sequence length
        # could vary or if it's dynamically determined
        x = self.positional_encoding(x)
        x = self.transformer_encoder(x)  # Pass through transformer encoder
        return x


Setting up device

In [29]:
# Check if CUDA (NVIDIA GPU) is available, then use it; otherwise, fall back to CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# For PyTorch 1.12 or newer, to include support for Apple Silicon (MPS)
if torch.backends.mps.is_available():
    device = torch.device('mps')  # Use Apple's Metal Performance Shaders (MPS)
elif torch.cuda.is_available():
    device = torch.device('cuda')  # Use NVIDIA GPU with CUDA
else:
    device = torch.device('cpu')  # Use CPU

Testing Image Encoder

In [33]:
# Parameters
num_patches = (448 // 16) * (448 // 16)  # Assuming the image is resized to 448x448 and patches are 16x16
embed_dim = 768  # Assuming each patch is projected to an embedding dimension of 768

# Initialize the encoder
image_encoder = ModifiedImageEncoder(
    num_patches=num_patches,
    embed_dim=embed_dim,
    num_layers=6,
    num_heads=12,
    d_ff=2048,
    dropout=0.1,
    batch_first = True
)

# Assuming the device setup from the previous discussion
image_encoder = image_encoder.to(device)
patch_vectors_tensor = patch_vectors_tensor.to(device)

# Encode the preprocessed patches
with torch.no_grad():
    encoded_patches = image_encoder(patch_vectors_tensor.unsqueeze(0))  # Add a batch dimension

print("Encoded patches shape:", encoded_patches.shape)


TypeError: ModifiedImageEncoder.__init__() got an unexpected keyword argument 'batch_first'